In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import timedelta

# PARAMETERS
tick1 = "DIS"          # ticker we are wanting for earnings on 
tick2 = "FOXA"           # ticker we have earnings and close on
market_ticker = 'SPY'   # market proxy
start = "2022-10-14"    # 3 years back from 2025-10-14
end = "2025-10-14"

# 1) Download daily prices with retry logic
print("Downloading data...")
import time

max_retries = 3
retry_delay = 5  # seconds

for attempt in range(max_retries):
    try:
        data1 = yf.download([tick1, tick2, market_ticker], start=start, end=end, progress=False, timeout=30)
        if not data1.empty:
            data1 = data1.dropna()
            print(f"✓ Data downloaded successfully")
            break
    except Exception as e:
        print(f"Attempt {attempt + 1}/{max_retries} failed: {e}")
        if attempt < max_retries - 1:
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        else:
            print("❌ Failed to download data after all retries")
            print("Possible solutions:")
            print("1. Check your internet connection")
            print("2. Try again in a few minutes (Yahoo Finance may be rate limiting)")
            print("3. Use a VPN if Yahoo Finance is blocked")
            raise

# 2) Compute daily log returns
rets_close = data1['Close'][[tick1, tick2, market_ticker]]
rets = np.log(rets_close / rets_close.shift(1)).dropna()

# DAILY RETURN CORRELATION (Pearson)
daily_corr = rets[tick1].corr(rets[tick2])
print(f"\n{'='*80}")
print(f"{'STOCK CORRELATION ANALYSIS':^80}")
print(f"{'='*80}")
print(f"\n  {tick1} vs {tick2} Daily Log-Return Correlation: {daily_corr:.4f}")
print(f"\n  ℹ️  Correlation Interpretation:")
if daily_corr > 0.7:
    print(f"     STRONG positive correlation - stocks move together closely")
    print(f"     When one goes up, the other typically goes up too")
elif daily_corr > 0.4:
    print(f"     MODERATE positive correlation - some tendency to move together")
elif daily_corr > 0:
    print(f"     WEAK positive correlation - slight tendency to move together")
else:
    print(f"     Negative or no correlation - stocks move independently")
print(f"\n{'='*80}\n")

# 3) Get earnings dates and EPS data
print("Fetching earnings dates and EPS data...")
tick1_obj = yf.Ticker(tick1)
tick2_obj = yf.Ticker(tick2)
ed1 = tick1_obj.earnings_dates
ed2 = tick2_obj.earnings_dates

# Convert to DataFrame with date index and include EPS surprise data
def clean_earnings(earn_df, start_date, end_date):
    if earn_df is None or len(earn_df) == 0:
        return pd.DataFrame(columns=['date', 'eps_actual', 'eps_estimate', 'eps_surprise'])
    
    try:
        df = earn_df.copy()
        # Convert index to datetime and extract date
        df.index = pd.to_datetime(df.index)
        
        # Create clean dataframe with EPS data
        df_clean = pd.DataFrame({
            'date': df.index.date,
            'eps_actual': df.get('Reported EPS', pd.Series([None]*len(df))),
            'eps_estimate': df.get('EPS Estimate', pd.Series([None]*len(df)))
        })
        
        # Calculate EPS surprise (actual - estimate)
        df_clean['eps_surprise'] = df_clean['eps_actual'] - df_clean['eps_estimate']
        df_clean['beat_estimate'] = df_clean['eps_surprise'] > 0
        
        df_clean = df_clean.drop_duplicates(subset=['date'])
        
        # Filter to date range
        start_dt = pd.to_datetime(start_date).date()
        end_dt = pd.to_datetime(end_date).date()
        df_clean = df_clean[(df_clean['date'] >= start_dt) & (df_clean['date'] <= end_dt)]
        
        return df_clean.sort_values('date').reset_index(drop=True)
    except Exception as e:
        print(f"Error cleaning earnings: {e}")
        return pd.DataFrame(columns=['date', 'eps_actual', 'eps_estimate', 'eps_surprise'])

ed1 = clean_earnings(ed1, start, end)
ed2 = clean_earnings(ed2, start, end)

print(f"\n{tick1} earnings dates found: {len(ed1)}")
print(f"{tick2} earnings dates found: {len(ed2)}")

# Event study function
def event_abnormal_returns(ticker, earnings_dates, prices_df, market_series, window=1):
    """
    window: number of trading days before/after for CAR
    Returns DataFrame with AR (day 0), CAR, EPS data, and actual price movements
    """
    results = []
    
    # Get full price data (OHLC)
    full_data = data1.copy()
    
    for idx, row in earnings_dates.iterrows():
        d = row['date']
        d_ts = pd.to_datetime(d)
        all_dates = prices_df.index
        
        # Find first trading date >= earnings date
        candidates = all_dates[all_dates >= d_ts]
        if len(candidates) == 0:
            continue
        event_day = candidates[0]
        
        # Estimation window: prior 120 trading days ending 2 days before event
        est_end = event_day - pd.Timedelta(days=2)
        est_start = est_end - pd.Timedelta(days=180)  # ~120 trading days
        est_window = prices_df.loc[(prices_df.index >= est_start) & (prices_df.index <= est_end)]
        
        if len(est_window) < 60:
            continue
        
        # Compute returns on estimation window
        r_stock = np.log(prices_df.loc[est_window.index] / prices_df.shift(1).loc[est_window.index]).dropna()
        r_mkt = np.log(market_series.loc[est_window.index] / market_series.shift(1).loc[est_window.index]).dropna()
        
        # Align data
        df_est = pd.concat([r_stock, r_mkt], axis=1).dropna()
        if len(df_est) < 30:
            continue
        
        # Market model regression
        X = sm.add_constant(df_est.iloc[:, 1])
        model = sm.OLS(df_est.iloc[:, 0], X).fit()
        alpha, beta = model.params.iloc[0], model.params.iloc[1]
        
        # Calculate AR on event day
        if event_day not in prices_df.index:
            continue
        
        pos = prices_df.index.get_loc(event_day)
        if pos == 0:
            continue
        
        prev_day = prices_df.index[pos - 1]
        r_event = np.log(prices_df.loc[event_day] / prices_df.loc[prev_day])
        r_mkt_event = np.log(market_series.loc[event_day] / market_series.loc[prev_day])
        ar0 = r_event - (alpha + beta * r_mkt_event)
        
        # CAR over window
        start_pos = max(0, pos - window)
        end_pos = min(len(prices_df) - 1, pos + window)
        
        car = 0.0
        for p in range(start_pos + 1, end_pos + 1):
            cur = prices_df.index[p]
            prev = prices_df.index[p - 1]
            r_cur = np.log(prices_df.loc[cur] / prices_df.loc[prev])
            r_mkt_cur = np.log(market_series.loc[cur] / market_series.loc[prev])
            ar_cur = r_cur - (alpha + beta * r_mkt_cur)
            car += ar_cur
        
        # Calculate actual price movements for trading
        # Get next trading day after event day
        next_pos = min(len(prices_df) - 1, pos + 1)
        next_day = prices_df.index[next_pos] if next_pos < len(prices_df) else None
        
        # Metric 1: Close(day prior) to Open(next tradable day)
        # This captures overnight reaction to earnings
        close_prior_to_open_next = None
        if next_day is not None and ticker in full_data['Close'].columns and ticker in full_data['Open'].columns:
            try:
                close_prior = full_data['Close'][ticker].loc[prev_day]
                open_next = full_data['Open'][ticker].loc[next_day]
                close_prior_to_open_next = (open_next - close_prior) / close_prior
            except:
                pass
        
        # Metric 2: Close(day prior) to Close(day of)
        # This captures the full day's price movement
        close_prior_to_close_of = None
        try:
            close_prior = prices_df.loc[prev_day]
            close_of = prices_df.loc[event_day]
            close_prior_to_close_of = (close_of - close_prior) / close_prior
        except:
            pass
        
        # Metric 3: Open(day of) to Close(day of)
        # This captures intraday movement on tradable day
        open_to_close_of = None
        if ticker in full_data['Open'].columns:
            try:
                open_of = full_data['Open'][ticker].loc[event_day]
                close_of = full_data['Close'][ticker].loc[event_day]
                open_to_close_of = (close_of - open_of) / open_of
            except:
                pass
        
        results.append({
            "earnings_date": event_day.date(),
            "AR0": ar0,
            f"CAR(-{window},{window})": car,
            "alpha": alpha,
            "beta": beta,
            "eps_actual": row.get('eps_actual', None),
            "eps_estimate": row.get('eps_estimate', None),
            "eps_surprise": row.get('eps_surprise', None),
            "beat_estimate": row.get('beat_estimate', None),
            "close_prior_to_open_next": close_prior_to_open_next,
            "close_prior_to_close_of": close_prior_to_close_of,
            "open_to_close_of": open_to_close_of
        })
    
    return pd.DataFrame(results)

# Prepare data for event study
prices_df1 = data1['Close'][tick1]
prices_df2 = data1['Close'][tick2]
mkt = data1['Close'][market_ticker]

# Run event studies
print(f"\nRunning event study for {tick1}...")
results1 = event_abnormal_returns(tick1, ed1, prices_df1, mkt, window=1)

print(f"Running event study for {tick2}...")
results2 = event_abnormal_returns(tick2, ed2, prices_df2, mkt, window=1)

# Display results
print(f"\n{'='*80}")
print(f"{f'{tick1} EARNINGS EVENT STUDY RESULTS':^80}")
print(f"{'='*80}")
print(f"\n  ℹ️  What These Metrics Mean:")
print(f"     • AR0 = Abnormal Return on earnings day (actual return - expected return)")
print(f"     • CAR(-1,+1) = Total abnormal return over 3-day window (day before, day of, day after)")
print(f"     • Alpha = Stock's baseline return independent of market")
print(f"     • Beta = Stock's sensitivity to market moves (>1 means more volatile than market)")
print(f"     • Close Prior → Open Next = Overnight gap (pre-market reaction)")
print(f"     • Close Prior → Close Of = Full day move (total tradable reaction)")
print(f"     • Open → Close Of = Intraday move (reaction during trading hours)")
print(f"\n{'─'*80}\n")
if len(results1) > 0:
    # Display main results
    display_cols = ['earnings_date', 'AR0', 'CAR(-1,1)', 'alpha', 'beta', 
                   'close_prior_to_open_next', 'close_prior_to_close_of', 'open_to_close_of',
                   'eps_actual', 'eps_estimate', 'beat_estimate']
    # Only show columns that exist
    display_cols = [col for col in display_cols if col in results1.columns]
    print(f"{results1[display_cols].to_string(index=False)}\n")
    print(f"{'─'*80}")
    print(f"  📊 Summary Statistics:")
    print(f"     • Average AR(0):        {results1['AR0'].mean():>8.4f}  ({results1['AR0'].mean()*100:>6.2f}%)")
    print(f"       → Typical market reaction to {tick1} earnings")
    print(f"     • Average CAR(-1,+1):   {results1['CAR(-1,1)'].mean():>8.4f}  ({results1['CAR(-1,1)'].mean()*100:>6.2f}%)")
    print(f"       → Full 3-day earnings impact")
    
    # Add price movement statistics
    if 'close_prior_to_open_next' in results1.columns:
        avg_overnight = results1['close_prior_to_open_next'].mean()
        print(f"     • Avg Overnight Gap:    {avg_overnight:>8.4f}  ({avg_overnight*100:>6.2f}%)")
        print(f"       → Average pre-market reaction")
    if 'close_prior_to_close_of' in results1.columns:
        avg_full = results1['close_prior_to_close_of'].mean()
        print(f"     • Avg Full Day Move:    {avg_full:>8.4f}  ({avg_full*100:>6.2f}%)")
        print(f"       → Average total tradable move")
    if 'open_to_close_of' in results1.columns:
        avg_intraday = results1['open_to_close_of'].mean()
        print(f"     • Avg Intraday Move:    {avg_intraday:>8.4f}  ({avg_intraday*100:>6.2f}%)")
        print(f"       → Average trading hours reaction")
    
    print(f"     • Median AR(0):         {results1['AR0'].median():>8.4f}  ({results1['AR0'].median()*100:>6.2f}%)")
    print(f"       → Middle value (not affected by outliers)")
    print(f"     • Std Dev AR(0):        {results1['AR0'].std():>8.4f}")
    print(f"       → Volatility/unpredictability of earnings reactions")
    print(f"     • Positive Events:      {(results1['AR0'] > 0).sum():>8d} / {len(results1)}")
    print(f"       → How often {tick1} beats expectations")
    print(f"{'─'*80}\n")
else:
    print("  ❌ No valid events found\n")

print(f"\n{'='*80}")
print(f"{f'{tick2} EARNINGS EVENT STUDY RESULTS':^80}")
print(f"{'='*80}")
print(f"\n  ℹ️  What These Metrics Mean:")
print(f"     • AR0 = Abnormal Return on earnings day (actual return - expected return)")
print(f"     • CAR(-1,+1) = Total abnormal return over 3-day window (day before, day of, day after)")
print(f"     • Alpha = Stock's baseline return independent of market")
print(f"     • Beta = Stock's sensitivity to market moves (>1 means more volatile than market)")
print(f"     • Close Prior → Open Next = Overnight gap (pre-market reaction)")
print(f"     • Close Prior → Close Of = Full day move (total tradable reaction)")
print(f"     • Open → Close Of = Intraday move (reaction during trading hours)")
print(f"\n{'─'*80}\n")
if len(results2) > 0:
    # Display main results
    display_cols = ['earnings_date', 'AR0', 'CAR(-1,1)', 'alpha', 'beta', 
                   'close_prior_to_open_next', 'close_prior_to_close_of', 'open_to_close_of',
                   'eps_actual', 'eps_estimate', 'beat_estimate']
    # Only show columns that exist
    display_cols = [col for col in display_cols if col in results2.columns]
    print(f"{results2[display_cols].to_string(index=False)}\n")
    print(f"{'─'*80}")
    print(f"  📊 Summary Statistics:")
    print(f"     • Average AR(0):        {results2['AR0'].mean():>8.4f}  ({results2['AR0'].mean()*100:>6.2f}%)")
    print(f"       → Typical market reaction to {tick2} earnings")
    print(f"     • Average CAR(-1,+1):   {results2['CAR(-1,1)'].mean():>8.4f}  ({results2['CAR(-1,1)'].mean()*100:>6.2f}%)")
    print(f"       → Full 3-day earnings impact")
    
    # Add price movement statistics
    if 'close_prior_to_open_next' in results2.columns:
        avg_overnight = results2['close_prior_to_open_next'].mean()
        print(f"     • Avg Overnight Gap:    {avg_overnight:>8.4f}  ({avg_overnight*100:>6.2f}%)")
        print(f"       → Average pre-market reaction")
    if 'close_prior_to_close_of' in results2.columns:
        avg_full = results2['close_prior_to_close_of'].mean()
        print(f"     • Avg Full Day Move:    {avg_full:>8.4f}  ({avg_full*100:>6.2f}%)")
        print(f"       → Average total tradable move")
    if 'open_to_close_of' in results2.columns:
        avg_intraday = results2['open_to_close_of'].mean()
        print(f"     • Avg Intraday Move:    {avg_intraday:>8.4f}  ({avg_intraday*100:>6.2f}%)")
        print(f"       → Average trading hours reaction")
    
    print(f"     • Median AR(0):         {results2['AR0'].median():>8.4f}  ({results2['AR0'].median()*100:>6.2f}%)")
    print(f"       → Middle value (not affected by outliers)")
    print(f"     • Std Dev AR(0):        {results2['AR0'].std():>8.4f}")
    print(f"       → Volatility/unpredictability of earnings reactions")
    print(f"     • Positive Events:      {(results2['AR0'] > 0).sum():>8d} / {len(results2)}")
    print(f"       → How often {tick2} beats expectations")
    print(f"{'─'*80}\n")
else:
    print("  ❌ No valid events found\n")

print(f"\n{'='*80}")
print(f"{'COMPARATIVE ANALYSIS':^80}")
print(f"{'='*80}")
print(f"\n  ℹ️  This section compares overall earnings performance between the two companies")
print(f"{'─'*80}\n")
if len(results1) > 0 and len(results2) > 0:
    print(f"  📈 Earnings Reaction Comparison:")
    print(f"     • {tick1} Avg AR:  {results1['AR0'].mean():>8.4f} ({results1['AR0'].mean()*100:>6.2f}%)")
    print(f"     • {tick2} Avg AR:  {results2['AR0'].mean():>8.4f} ({results2['AR0'].mean()*100:>6.2f}%)")
    print(f"     • Difference:     {(results1['AR0'].mean() - results2['AR0'].mean()):>8.4f} ({(results1['AR0'].mean() - results2['AR0'].mean())*100:>6.2f}%)")
    
    winner = tick1 if results1['AR0'].mean() > results2['AR0'].mean() else tick2
    print(f"\n  🏆 {winner} shows stronger positive earnings reaction on average")
    
    print(f"\n  📊 Average Beta (Market Sensitivity):")
    print(f"     • {tick1}: {results1['beta'].mean():.3f} → ", end="")
    if results1['beta'].mean() > 1.3:
        print(f"Much more volatile than market")
    elif results1['beta'].mean() > 1.1:
        print(f"More volatile than market")
    else:
        print(f"Similar volatility to market")
    print(f"     • {tick2}: {results2['beta'].mean():.3f} → ", end="")
    if results2['beta'].mean() > 1.3:
        print(f"Much more volatile than market")
    elif results2['beta'].mean() > 1.1:
        print(f"More volatile than market")
    else:
        print(f"Similar volatility to market")
    
    # EARNINGS CO-MOVEMENT ANALYSIS
    print(f"\n{'='*80}")
    print(f"{'EARNINGS CO-MOVEMENT ANALYSIS':^80}")
    print(f"{'='*80}")
    print(f"\n  Question: Does positive {tick2} earnings predict positive {tick1}?")
    print(f"{'─'*80}\n")
    
    # Find overlapping/nearby earnings dates (within 7 days)
    overlapping_events = []
    for idx1, row1 in results1.iterrows():
        date1 = pd.to_datetime(row1['earnings_date'])
        for idx2, row2 in results2.iterrows():
            date2 = pd.to_datetime(row2['earnings_date'])
            days_apart = abs((date2 - date1).days)
            if days_apart <= 7:  # Within 1 week
                overlapping_events.append({
                    f'{tick1}_date': date1.date(),
                    f'{tick2}_date': date2.date(),
                    'days_apart': days_apart,
                    f'{tick1}_beat': row1.get('beat_estimate', None),
                    f'{tick2}_beat': row2.get('beat_estimate', None),
                    f'{tick1}_AR': row1['AR0'],
                    f'{tick2}_AR': row2['AR0'],
                    'same_sign_AR': (row1['AR0'] > 0) == (row2['AR0'] > 0),
                    'both_beat': (row1.get('beat_estimate', False) and row2.get('beat_estimate', False)) if (pd.notna(row1.get('beat_estimate')) and pd.notna(row2.get('beat_estimate'))) else None
                })
    
    if overlapping_events:
        overlap_df = pd.DataFrame(overlapping_events)
        print(f"  📅 Earnings within 7 days: {len(overlap_df)} events\n")
        
        # Display with EPS beat/miss data
        display_cols = [f'{tick1}_date', f'{tick2}_date', 'days_apart', 
                       f'{tick1}_beat', f'{tick2}_beat', 
                       f'{tick1}_AR', f'{tick2}_AR']
        print(overlap_df[display_cols].to_string(index=False))
        
        same_direction_AR = overlap_df['same_sign_AR'].sum()
        total = len(overlap_df)
        pct_AR = (same_direction_AR / total * 100) if total > 0 else 0
        
        # Count earnings beat co-movement
        valid_eps = overlap_df['both_beat'].notna()
        if valid_eps.sum() > 0:
            both_beat_or_miss = overlap_df[valid_eps].apply(
                lambda x: (x[f'{tick1}_beat'] == x[f'{tick2}_beat']), axis=1
            ).sum()
            total_eps = valid_eps.sum()
            pct_eps = (both_beat_or_miss / total_eps * 100)
            
            both_beat_count = overlap_df[valid_eps]['both_beat'].sum()
        else:
            both_beat_or_miss = None
            total_eps = 0
            pct_eps = None
            both_beat_count = 0
        
        print(f"\n  {'─'*80}")
        print(f"  📊 PART 1: ACTUAL EARNINGS QUALITY (Beat/Miss Estimates)")
        if total_eps >= 3:
            print(f"     Both Beat/Miss Same:   {both_beat_or_miss}/{total_eps} events    ({pct_eps:.1f}%)")
            print(f"     Both Beat Together:    {both_beat_count} events")
            print(f"\n     → When {tick2} beats estimates, {tick1} moves same direction {pct_eps:.0f}% of time")
        else:
            print(f"     ⚠️  Insufficient EPS data (only {total_eps} events with EPS)")
        
        print(f"\n  {'─'*80}")
        print(f"  📊 PART 2: STOCK PRICE REACTIONS (Abnormal Returns on Earnings Day)")
        print(f"     Same Direction AR0:    {same_direction_AR}/{total} events    ({pct_AR:.1f}%)")
        print(f"       → Both stocks had same direction AR on their respective earnings days")
        
        if total >= 3:
            ar_corr = overlap_df[f'{tick1}_AR'].corr(overlap_df[f'{tick2}_AR'])
            print(f"     AR0 Correlation:       {ar_corr:>6.4f}")
            print(f"     Avg {tick1} AR0:       {overlap_df[f'{tick1}_AR'].mean():>6.4f}    ({overlap_df[f'{tick1}_AR'].mean()*100:>5.2f}%)")
            print(f"     Avg {tick2} AR0:       {overlap_df[f'{tick2}_AR'].mean():>6.4f}    ({overlap_df[f'{tick2}_AR'].mean()*100:>5.2f}%)")
            print(f"\n     → When {tick2} has positive earnings day reaction, {tick1} also positive {pct_AR:.0f}% of time")
            
            # Combined analysis
            print(f"\n  {'─'*80}")
            print(f"  📊 PART 3: COMBINED - EPS BEAT → EARNINGS DAY STOCK REACTION")
            
            if total_eps >= 3:
                # When tick2 beats, does tick1 also beat AND have positive AR?
                tick2_beats = overlap_df[overlap_df[f'{tick2}_beat'] == True]
                if len(tick2_beats) > 0:
                    tick1_also_beat = tick2_beats[f'{tick1}_beat'].sum()
                    tick1_positive_ar = (tick2_beats[f'{tick1}_AR'] > 0).sum()
                    
                    print(f"     When {tick2} BEATS estimates ({len(tick2_beats)} times):")
                    print(f"       • {tick1} also beats:       {tick1_also_beat}/{len(tick2_beats)} ({tick1_also_beat/len(tick2_beats)*100:.0f}%)")
                    print(f"       • {tick1} positive AR0:     {tick1_positive_ar}/{len(tick2_beats)} ({tick1_positive_ar/len(tick2_beats)*100:.0f}%)")
                    print(f"         (on {tick1}'s own earnings day)")
                
                # When tick2 misses
                tick2_misses = overlap_df[overlap_df[f'{tick2}_beat'] == False]
                if len(tick2_misses) > 0:
                    tick1_also_miss = (tick2_misses[f'{tick1}_beat'] == False).sum()
                    tick1_negative_ar = (tick2_misses[f'{tick1}_AR'] < 0).sum()
                    
                    print(f"\n     When {tick2} MISSES estimates ({len(tick2_misses)} times):")
                    print(f"       • {tick1} also misses:      {tick1_also_miss}/{len(tick2_misses)} ({tick1_also_miss/len(tick2_misses)*100:.0f}%)")
                    print(f"       • {tick1} negative AR0:     {tick1_negative_ar}/{len(tick2_misses)} ({tick1_negative_ar/len(tick2_misses)*100:.0f}%)")
                    print(f"         (on {tick1}'s own earnings day)")
            else:
                print(f"     ⚠️  Insufficient EPS data for combined analysis")
            
            print(f"\n  {'─'*80}")
            print(f"  🎯 FINAL ANSWER:")
            
            # Determine strength
            eps_strong = (total_eps >= 3 and pct_eps >= 70)
            ar_strong = (ar_corr > 0.5 and pct_AR >= 70)
            
            if eps_strong and ar_strong:
                print(f"     ✅ YES - Strong Predictive Power on BOTH")
                print(f"        EPS Co-movement: {pct_eps:.0f}% | AR0 Correlation: {ar_corr:.2f}")
                print(f"        {tick2} earnings quality predicts {tick1} earnings quality AND earnings day reactions")
            elif eps_strong:
                print(f"     ✅ YES - Strong EPS Prediction, Moderate Earnings Day Reaction")
                print(f"        EPS Co-movement: {pct_eps:.0f}% | AR0 Correlation: {ar_corr:.2f}")
                print(f"        {tick2} beats/misses predict {tick1} beats/misses reliably")
            elif ar_strong:
                print(f"     ⚠️  MODERATE - Earnings Day Prices Move Together, But EPS Unclear")
                print(f"        AR0 Correlation: {ar_corr:.2f} | EPS data: {total_eps} events")
                print(f"        Earnings day reactions correlate, but earnings quality connection weak")
            else:
                print(f"     ❌ WEAK - Limited Predictive Power")
                print(f"        AR0 Correlation: {ar_corr:.2f} | Same Direction: {pct_AR:.0f}%")
        else:
            print(f"\n  ⚠️  Insufficient data ({total} events) - need at least 3 for analysis")
    else:
        print(f"  ℹ️  No concurrent earnings (within 7 days)")
    
    # Overall pattern
    print(f"\n  {'─'*80}")
    print(f"  📈 OVERALL PATTERN (All Quarters):")
    
    # Stock reactions
    pos_tick2 = (results2['AR0'] > 0).sum()
    pos_tick1 = (results1['AR0'] > 0).sum()
    total_tick2 = len(results2)
    total_tick1 = len(results1)
    
    # EPS beats
    beats_tick2 = results2['beat_estimate'].sum() if 'beat_estimate' in results2.columns else None
    beats_tick1 = results1['beat_estimate'].sum() if 'beat_estimate' in results1.columns else None
    
    print(f"     {tick2} Beat Estimates:    {beats_tick2}/{total_tick2} events     ({beats_tick2/total_tick2*100:>5.1f}%)" if beats_tick2 else f"     {tick2} Beat Estimates:    N/A")
    print(f"     {tick1} Beat Estimates:    {beats_tick1}/{total_tick1} events     ({beats_tick1/total_tick1*100:>5.1f}%)" if beats_tick1 else f"     {tick1} Beat Estimates:    N/A")
    print(f"     {tick2} Positive AR:        {pos_tick2}/{total_tick2} events     ({pos_tick2/total_tick2*100:>5.1f}%)")
    print(f"     {tick1} Positive AR:        {pos_tick1}/{total_tick1} events     ({pos_tick1/total_tick1*100:>5.1f}%)")
    print(f"       → Percentage of earnings days with positive abnormal returns")
    
    if len(results1) == len(results2):
        overall_corr = results1['AR0'].corr(results2['AR0'])
        print(f"     Quarterly AR Corr:      {overall_corr:>6.4f}")
    
    # Trading implications
    print(f"\n  {'='*80}")
    print(f"  💰 TRADING SIGNAL STRENGTH:")
    print(f"  {'='*80}")
    if overlapping_events and len(overlap_df) >= 3:
        ar_corr = overlap_df[f'{tick1}_AR'].corr(overlap_df[f'{tick2}_AR'])
        same_pct = (overlap_df['same_sign_AR'].sum() / len(overlap_df) * 100)
        
        print(f"     AR0 Correlation:    {ar_corr:>5.3f}  (earnings day reactions)")
        print(f"     Same Direction AR0: {same_pct:>5.1f}%")
        if total_eps >= 3:
            print(f"     EPS Co-movement:    {pct_eps:>5.1f}%")
        print(f"     Data Points:        {len(overlap_df):>5d} concurrent earnings")
        print()
        
        if ar_corr > 0.5 and same_pct >= 70:
            print(f"     ✅ TRADEABLE - Use {tick2} earnings day reaction as indicator for {tick1}")
            if total_eps >= 3 and pct_eps >= 70:
                print(f"        Strong on BOTH earnings quality and earnings day stock price")
        elif ar_corr > 0.3:
            print(f"     ⚠️  MARGINAL - Weak signal, high risk")
        else:
            print(f"     ❌ NOT TRADEABLE - No reliable relationship")
    else:
        print(f"     ⚠️  INSUFFICIENT DATA for trading signals")
    print(f"  {'─'*80}\n")
    
print(f"\n{'='*80}")
print(f"{'✓ ANALYSIS COMPLETE':^80}")
print(f"{'='*80}\n")

✓ Data downloaded successfully

                           STOCK CORRELATION ANALYSIS                           

  DIS vs FOXA Daily Log-Return Correlation: 0.4278

  ℹ️  Correlation Interpretation:
     MODERATE positive correlation - some tendency to move together


Fetching earnings dates and EPS data...


C:\Users\hleiber2022\AppData\Local\Temp\ipykernel_22104\1886852877.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data1 = yf.download([tick1, tick2, market_ticker], start=start, end=end, progress=False, timeout=30)



DIS earnings dates found: 12
FOXA earnings dates found: 12

Running event study for DIS...
Running event study for FOXA...

                        DIS EARNINGS EVENT STUDY RESULTS                        

  ℹ️  What These Metrics Mean:
     • AR0 = Abnormal Return on earnings day (actual return - expected return)
     • CAR(-1,+1) = Total abnormal return over 3-day window (day before, day of, day after)
     • Alpha = Stock's baseline return independent of market
     • Beta = Stock's sensitivity to market moves (>1 means more volatile than market)
     • Close Prior → Open Next = Overnight gap (pre-market reaction)
     • Close Prior → Close Of = Full day move (total tradable reaction)
     • Open → Close Of = Intraday move (reaction during trading hours)

────────────────────────────────────────────────────────────────────────────────

earnings_date       AR0  CAR(-1,1)     alpha     beta  close_prior_to_open_next  close_prior_to_close_of  open_to_close_of  eps_actual  eps_estimate